In [1]:
import pandas as pd
import numpy as np
import json
from urllib import request

Class Roster

In [ ]:
def getFineData(row): return row['enrollGroups'][0]['classSections'][0]['meetings']
def getProf(row): 
  x = lambda r: r[0]['instructors'][0]['firstName'].strip() + ' '+r[0]['instructors'][0]['lastName'].strip()
  try: 
    return x(row)
  except: return ""
def getData(fineData, obj): 
  try:
    x = lambda r: r[0][obj]
    return x(fineData)
  except: return ""

In [ ]:
def scrape_class_roster(sem, csv_name):
  subjects_url = 'https://classes.cornell.edu/api/2.0/config/subjects.json?roster='+sem
  subjects_json = json.loads(request.urlopen(subjects_url).read())
  subject_list = [x['value'] for x in subjects_json['data']['subjects']]
  courses = pd.DataFrame(columns = ["Dept", "Number", "Course_Name", "Professor","Start_Time", "End_Time", "Location","Description","Pre/coreqs","Offered"])

  for subject in subject_list: 
    url = "https://classes.cornell.edu/api/2.0/search/classes.json?roster="+sem+f"&subject={subject}"
    data_json = json.loads(request.urlopen(url).read())
    df = pd.DataFrame(data_json['data']['classes'])

    df['prof'] = df.apply(lambda row: getProf( getFineData(row)), axis=1)
    df['startTime'] = df.apply(lambda row: getData(getFineData(row), 'timeStart'), axis=1)  
    df['endTime'] = df.apply(lambda row: getData(getFineData(row), 'timeEnd'), axis=1)  
    df['location'] = df.apply(lambda row: getData(getFineData(row), 'facilityDescr'), axis=1)  

    df = df[["subject", "catalogNbr", "titleShort", "prof", "startTime", "endTime", "location", "description", "catalogPrereqCoreq", "catalogWhenOffered"]]
    df = df.rename(columns={"subject": "Dept", 'catalogNbr':"Number", "titleShort":"Course_Name", "prof":"Professor", "startTime":"Start_Time", "endTime":"End_Time", "location": "Location", "description":"Description", "catalogPrereqCoreq":"Pre/coreqs", "catalogWhenOffered":"Offered"})

    courses = courses.merge(df, how='outer') 

  courses.to_csv('data/class_roster_'+sem+'.csv')


In [114]:
sem = "SP22"
scrape_class_roster(sem)

In [170]:
sem = "FA22"
scrape_class_roster(sem)

CUreviews

In [8]:
def scrape_cureviews (courses):
  url = 'https://www.cureviews.org/v2/getCourseByInfo/'
  courses['Dept + Number'] = courses.Dept + " " + courses.Number.map(str)

  class_diff = {}
  rating = {}
  workload = {}

  for index, row in courses['Dept + Number'].iteritems():
    try:
      result = request.post(url, json={'number':int(row[2]), 'subject':row[1].lower}).json()['result']
      class_diff[row] = result['classDifficulty']
      rating[row] = result['classRating']
      workload[row] = result['classWorkload']
    except:
      class_diff[row] = ""
      rating[row] = ""
      workload[row] = ""
  
  courses["class_diff"] = class_diff
  courses["rating"] = rating
  courses["workload"] = workload

  return courses

In [9]:
courses_FA = scrape_cureviews(pd.read_csv('data/class_roster_FA22.csv'))
courses_SP = scrape_cureviews(pd.read_csv('data/class_roster_SP22.csv'))

In [10]:
courses = courses_FA.merge(courses_SP, how='outer')

Rate My Prof

In [2]:
# from https://github.com/tisuela/ratemyprof-api/tree/master/ratemyprof_api
from professor import *
from ratemyprof_api import *

In [3]:
rmp = RateMyProfApi(298).scrape_professors()

In [6]:
num_ratings = {}
prof_rat = {}
prof_diff = {}
# count = 0
for prof in rmp:
    
    try:
        result = requests.get("https://www.ratemyprofessors.com/ShowRatings.jsp?tid=" + str(prof))
        page = result.content.decode('utf-8')

        num_ratings[rmp[prof].name] = rmp[prof].num_of_ratings
        prof_rat[rmp[prof].name] = rmp[prof].overall_rating
        diff = page.index('avgDifficulty')
        prof_diff[rmp[prof].name] = page[page.index(':', diff + 1)+1:page.index(',', diff + 1)]
        
    except:
        print(rmp[prof].name)
    # count += 1
    # if count > 10:
        # assert False

Aditya Manthri


In [7]:
profs = []
for prof in rmp:
    profs.append(rmp[prof].name)
prof_info = pd.DataFrame(columns = ['Professor', 'num_ratings', 'prof_rat', 'prof_diff'])
prof_info['Professor'] = profs

prof_info['num_ratings'] = prof_info['Professor'].map(num_ratings)
prof_info['prof_rat'] = prof_info['Professor'].map(prof_rat)
prof_info['prof_diff'] = prof_info['Professor'].map(prof_diff)
prof_info.to_csv('data/rmp.csv')

Concatenating medians

In [11]:
courses_w_prof = courses.merge(prof_info, on="Professor", how="outer")

In [16]:
medians = pd.read_csv('data/CornellMedianGrades.csv')
medians = medians[1:]

course_medians = courses_w_prof.merge(medians, on="Dept + Number", how="outer")

In [17]:
course_medians.to_csv('data/merged_data.csv')